Realizar histogram matching de acuerdo a:

https://github.com/CONABIO/antares3/blob/training-data-model-fit/madmex/lcc/bitemporal/distance.py#L5

# 1

Registrar productos de surfaces reflectances del año 95-96 (algunas escenas) disponibles en s3.

Se eligió el dc tile: (21,-19) definido de acuerdo al tiling en los archivos de configuración del satélite landsat de antares3.

Este dc tile corresponde al path/row: 032 042 (Durango) y de las escenas que están disponibles en s3 para el año 95-96 están:

LT050320421995020601T1-SC20180529121712/
LT050320421995031001T1-SC20180529121554/
LT050320421995041101T1-SC20180529121727/
LT050320421995051301T1-SC20180529121345/
LT050320421995100401T1-SC20180529121414/
LT050320421995102001T1-SC20180529120628/
LT050320421995112101T1-SC20180529121834/
LT050320421996010801T2-SC20180529123117/
LT050320421996012401T2-SC20180529122022/
LT050320421996031201T1-SC20180529122036/
LT050320421996041301T1-SC20180529121850/
LT050320421996042901T1-SC20180529121810/
LT050320421996051501T1-SC20180529122435/
LT050320421996102201T1-SC20180529123018/
LT050320421996110701T1-SC20180529122414/
LT050320421996112301T1-SC20180529121903/

Se eligen registrar:

LT050320421995020601T1-SC20180529121712/
LT050320421995031001T1-SC20180529121554/
LT050320421995112101T1-SC20180529121834/
LT050320421996012401T2-SC20180529122022/
LT050320421996031201T1-SC20180529122036/
LT050320421996110701T1-SC20180529122414/
LT050320421996112301T1-SC20180529121903/

y se copian a :

```
/LUSTRE/MADMEX/tasks/2019_tasks/histogram_matching/scenes_95_96/

/shared_volume/tasks/2019_tasks/histogram_matching/scenes_95_96/
```

Pasos de prepare, product add, dataset add e ingest:


```
antares prepare_metadata --path /shared_volume/tasks/2019_tasks/histogram_matching/scenes_95_96/ --dataset_name landsat_espa --outfile metadata_mex_l5.yaml -sc /shared_volume/scheduler.json

datacube -v product add ~/.config/madmex/indexing/ls5_espa_scenes.yaml

datacube -v dataset add metadata_mex_l5.yaml

datacube -v ingest -c ~/.config/madmex/ingestion/ls5_espa_mexico.yaml --executor distributed 172.17.0.2:8786
```


Check via cmd line

datacube product list

```
ls5_espa_mexico Landsat 5 Collection 1 processed to surface reflectance by espa. Resampled to 30m Mexico INEGI Lambert Conformal Conic projection with a 50 km tile size.
ls5_espa_scene  Landsat 5 Collection 1 Higher Level SR scene processed by ESPA. 30m UTM based projection.
```

# 2

Registrar los productos de las recetas (dc tile: (21,-19) para años 2014-2015) de acuerdo al notebook:

https://github.com/CONABIO/antares3-sandbox/blob/master/notebooks/ingest_recipe_products/ingest_recipe_products.ipynb

# 3 histogram matching

Uso de: https://github.com/CONABIO/antares3/blob/training-data-model-fit/madmex/lcc/bitemporal/distance.py#L5

Referencias:

https://github.com/mapbox/rio-hist

https://blog.mapbox.com/color-balancing-imagery-with-histogram-matching-be1b38c28509


## A) Cargar producto de años `source` (1995-1996)

In [1]:
%matplotlib inline
import matplotlib
from datacube.api import GridWorkflow

from madmex.wrappers import gwf_query
from madmex.util import join_dicts

In [2]:
product = ['ls5_espa_mexico']
begin = '1995-01-01'
end = '1996-12-31'
region = 'Durango'
gwf_kwargs = {'region': region, 'begin': begin, 'end': end}
dc_tile=(21,-19)
dict_list=[]

In [3]:
for prod in product:
    gwf_kwargs.update(product = prod)
    try:
        dict_list.append(gwf_query(**gwf_kwargs, view=False))
    # Exception is in case one of the product hasn't been registered in the datacube
    except Exception as e:
        pass
iterable = join_dicts(*dict_list, join='full').items()

In [4]:
lista_iter = list(iterable)

In [6]:
index = [x for x in range(0,len(lista_iter)) if lista_iter[x][0] == dc_tile]

In [7]:
var_source = lista_iter[index[0]]

In [8]:
var_source

((21, -19), [Tile<sources=<xarray.DataArray (time: 7)>
  array([(Dataset <id=979bee9a-40f0-4e12-bd21-86d0d2db3087 type=ls5_espa_mexico location=file:///shared_volume/datacube/datacube_ingest/LS5_espa/mexico/LS5_espa_21_-19_19950206164648000000.nc>,),
         (Dataset <id=4c9a2386-597c-405c-bbd9-f076e077ff71 type=ls5_espa_mexico location=file:///shared_volume/datacube/datacube_ingest/LS5_espa/mexico/LS5_espa_21_-19_19950310164531000000.nc>,),
         (Dataset <id=b817293f-7dda-48a8-a7d6-1a95dee6ab6d type=ls5_espa_mexico location=file:///shared_volume/datacube/datacube_ingest/LS5_espa/mexico/LS5_espa_21_-19_19951121163445000000.nc>,),
         (Dataset <id=37a54d8c-88e4-4883-8e41-d02830400be5 type=ls5_espa_mexico location=file:///shared_volume/datacube/datacube_ingest/LS5_espa/mexico/LS5_espa_21_-19_19960124163902000000.nc>,),
         (Dataset <id=98f4c96b-fbd2-4441-8195-6529852872df type=ls5_espa_mexico location=file:///shared_volume/datacube/datacube_ingest/LS5_espa/mexico/LS5_espa_

## B) Cargar producto de años `template` (2014-2015)

In [12]:
product = ['recipe_mex_L7L8_1415']
begin = '2014-01-01'
end = '2015-12-31'
region = 'Durango'
gwf_kwargs = {'region': region,  'begin': begin, 'end': end}
dc_tile=(21,-19)
dict_list=[]

In [13]:
for prod in product:
    gwf_kwargs.update(product = prod)
    try:
        dict_list.append(gwf_query(**gwf_kwargs, view=False))
    # Exception is in case one of the product hasn't been registered in the datacube
    except Exception as e:
        pass
iterable = join_dicts(*dict_list, join='full').items()

In [14]:
lista_iter_template = list(iterable)

In [15]:
index_template = [x for x in range(0,len(lista_iter_template)) if lista_iter_template[x][0] == dc_tile]

In [16]:
var_template = lista_iter_template[index_template[0]]

In [17]:
var_template

((21, -19), [Tile<sources=<xarray.DataArray (time: 1)>
  array([(Dataset <id=68d0425b-0a05-5455-a0f4-2bb2029abb86 type=recipe_mex_L7L8_1415 location=file://shared_volume/datacube/datacube_ingest/recipes/recipe_mex_L7L8_1415/madmex_003_21_-19_2014-12-31.nc>,)],
        dtype=object)
  Coordinates:
    * time     (time) datetime64[ns] 2014-12-31T12:00:00,
  	geobox=GeoBox(1667, 1667, Affine(30.0, 0.0, 2027370.0,
         0.0, -30.0, 1526540.0), PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]])>])

In [ ]:
sr_template = xr.auto_combine([GridWorkflow.load(x, dask_chunks={'x': 600, 'y': 600}) for x in var_template[1]], concat_dim='time')

In [ ]:
sr_template

## C) Masking de `source` (antes de cálculo de métricas en recetas)

In [26]:
import xarray as xr
from datacube.storage import masking

from madmex.util.xarray import to_float, to_int

In [19]:
crs = var_source[1][0].geobox.crs

In [21]:
sr_source = xr.auto_combine([GridWorkflow.load(x, dask_chunks={'x': 600, 'y': 600}) for x in var_source[1]], concat_dim='time')

In [23]:
sr_source.attrs['geobox'] = var_source[1][0].geobox

In [27]:
clear = masking.make_mask(sr_source.pixel_qa, cloud=False, cloud_shadow=False,
                          snow=False)
sr_source_1 = sr_source.where(clear)
sr_source_1 = sr_source_1.drop('pixel_qa')
sr_source_1 = sr_source_1.apply(func=to_float, keep_attrs=True)

In [28]:
sr_source_1

<xarray.Dataset>
Dimensions:  (time: 7, x: 1667, y: 1667)
Coordinates:
  * y        (y) float64 1.527e+06 1.526e+06 1.526e+06 ... 1.477e+06 1.477e+06
  * x        (x) float64 2.027e+06 2.027e+06 2.027e+06 ... 2.077e+06 2.077e+06
  * time     (time) datetime64[ns] 1995-02-06T16:46:48 ... 1996-11-23T16:56:17
Data variables:
    blue     (time, y, x) float64 dask.array<shape=(7, 1667, 1667), chunksize=(7, 600, 600)>
    green    (time, y, x) float64 dask.array<shape=(7, 1667, 1667), chunksize=(7, 600, 600)>
    red      (time, y, x) float64 dask.array<shape=(7, 1667, 1667), chunksize=(7, 600, 600)>
    nir      (time, y, x) float64 dask.array<shape=(7, 1667, 1667), chunksize=(7, 600, 600)>
    swir1    (time, y, x) float64 dask.array<shape=(7, 1667, 1667), chunksize=(7, 600, 600)>
    swir2    (time, y, x) float64 dask.array<shape=(7, 1667, 1667), chunksize=(7, 600, 600)>
Attributes:
    geobox:   GeoBox(Geometry({'type': 'Polygon', 'coordinates': [[(-106.7330...

## D) Histogram matching para cada banda de source tomando referencia bandas de template